In [51]:
# Importação das bibliotecas
import pandas as pd
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report, log_loss

In [52]:
# Dados tratados do IBOVESPA
dados_ibovespa = pd.read_excel("Dados Tratados - Ibovespa.xlsx")

In [53]:
# Limpeza e padronização dos nomes das colunas
dados_ibovespa.columns = dados_ibovespa.columns.str.strip().str.replace(' ', '_')

In [54]:
def converter_volume_str_to_num(valor): # Converte 'M' (Milhões) e 'B' (Bilhões) para a quantidade real de casas decimais
    valor = valor.replace('.', '').replace(',', '.') # Altera valores para estrutura de datatype 'float'
    if 'M' in valor:
        return float(valor.replace('M', '')) * 1e6
    elif 'B' in valor:
        return float(valor.replace('B', '')) * 1e9
    else:
        return float(valor)

dados_ibovespa['Vol.'] = dados_ibovespa['Vol.'].apply(converter_volume_str_to_num) # Aplica função na coluna 'Vol.'

In [55]:
dados_ibovespa['Data'] = pd.to_datetime(dados_ibovespa['Data'], dayfirst=True) # Transforma em datatype "datetime"

dados_ibovespa = dados_ibovespa.sort_values('Data') # Organiza os registros da coluna 'Data' começando do mais antigo

dados_ibovespa['Target'] = dados_ibovespa['Variacao'].shift(-1) # Cria uma coluna denominada 'Target', pega os valores da coluna 'Variacao' e desloca uma linha para cima, assim pegando o valor do dia atual para prever o do dia seguinte

dados_ibovespa = dados_ibovespa.dropna() # Remoção de registros nulos

dados_ibovespa['Target'] = dados_ibovespa['Target'].astype(int) # Transforma o datatype da coluna Target em int

In [56]:
# Fórmulas
dados_ibovespa['retorno_diario'] = (dados_ibovespa['Ultimo'] - dados_ibovespa['Abertura']) / dados_ibovespa['Abertura'] # Variação diária do valor negociado do índice IBOVESPA
dados_ibovespa['volatilidade'] = (dados_ibovespa['Maxima'] - dados_ibovespa['Minima']) / dados_ibovespa['Abertura'] # Variação relativa diária do índice Ibovespa em relação ao valor de abertura
dados_ibovespa['posicao_no_dia'] = (dados_ibovespa['Ultimo'] - dados_ibovespa['Minima']) / (dados_ibovespa['Maxima'] - dados_ibovespa['Minima'] + 1e-6) # Relação do fechamento diário (próximo de 1 = perto da máxima; próximo de 0 = perto da mínima)

dados_ibovespa['media_ultimos_3_dias'] = dados_ibovespa['Ultimo'].rolling(3).mean() # Média móvel dos últimos 3 dias do valor de fechamento
dados_ibovespa['media_ultimos_5_dias'] = dados_ibovespa['Ultimo'].rolling(5).mean() # Média móvel dos últimos 5 dias do valor de fechamento
dados_ibovespa['media_ultimos_10_dias'] = dados_ibovespa['Ultimo'].rolling(10).mean() # Média móvel dos últimos 10 dias do valor de fechamento
dados_ibovespa['media_exp_5_dias'] = dados_ibovespa['Ultimo'].ewm(span=5, adjust=False).mean() # Média exponencial dos últimos 5 dias do valor de fechamento
dados_ibovespa['media_exp_10_dias'] = dados_ibovespa['Ultimo'].ewm(span=10, adjust=False).mean() # Média exponencial dos últimos 10 dias do valor de fechamento

dados_ibovespa['retorno_3_dias'] = dados_ibovespa['Ultimo'].pct_change(3) # Retorno percentual dos últimos 3 dias

In [57]:
dados_ibovespa['Dia_da_Semana'] = dados_ibovespa['Dia_da_Semana'].astype('category').cat.codes # Substitui string dos dias da semana por índices numéricos

dados_ibovespa = dados_ibovespa.dropna() # Remoção de registros nulos (primeiros registros das fórmulas que calculam média baseando-se nos últimos)

In [58]:
# Divisão dos dados entre treino e teste
dados_treino = dados_ibovespa.iloc[:-30]
dados_teste = dados_ibovespa.iloc[-30:]

In [59]:
# Separação das features e do alvo no treino
features = dados_treino.drop(columns=['Data', 'Variacao', 'Target'])
alvo = dados_treino['Target']

In [60]:
# Configurar validação cruzada temporal
numero_folds = TimeSeriesSplit(n_splits=3)

accuracies = []
reports = []

fold = 1
for train_index, val_index in numero_folds.split(features):
    print(f"\nFold {fold}")
    features_train, features_val = features.iloc[train_index], features.iloc[val_index]
    alvo_train, alvo_val = alvo.iloc[train_index], alvo.iloc[val_index]

    model = lgb.LGBMClassifier(
        n_estimators=2000,
        learning_rate=0.01,
        num_leaves=60,
        max_depth=-1,
        class_weight='balanced',
        random_state=50,
        verbosity=-1
    )

    model.fit(
        features_train, alvo_train,
        eval_set=[(features_val, alvo_val)],
        eval_metric='binary_logloss',
        callbacks=[
            early_stopping(stopping_rounds=100),
            log_evaluation(period=100)
        ]
    )

    alvo_pred = model.predict(features_val)
    acc = accuracy_score(alvo_val, alvo_pred)
    print(f"Acurácia Fold {fold}: {acc:.5f}")

    accuracies.append(acc)
    reports.append(classification_report(alvo_val, alvo_pred, output_dict=True))

    fold += 1

# Preparar dados de teste
features_teste = dados_teste.drop(columns=['Data', 'Variacao', 'Target'])
alvo_teste = dados_teste['Target']

# Treinar modelo final com todos os dados de treino
modelo_teste = lgb.LGBMClassifier(
    n_estimators=2000,
    learning_rate=0.01,
    num_leaves=60,
    max_depth=-1,
    class_weight='balanced',
    random_state=50,
    verbosity=-1
)

modelo_teste.fit(features, alvo);

# Fazer predição probabilística no teste
pred_proba = modelo_teste.predict_proba(features_teste)[:, 1]
# Aplicar threshold customizado para decidir classes
threshold = 0.19
predicoes = (pred_proba >= threshold).astype(int)
# Avaliar resultados no teste
acc_final = accuracy_score(alvo_teste, predicoes)
print(f"\nAcurácia Final: {acc_final:.5f}")


Fold 1
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.545205
[200]	valid_0's binary_logloss: 0.552327
Early stopping, best iteration is:
[118]	valid_0's binary_logloss: 0.541232
Acurácia Fold 1: 0.74751

Fold 2
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.568809
[200]	valid_0's binary_logloss: 0.548577
[300]	valid_0's binary_logloss: 0.545009
Early stopping, best iteration is:
[259]	valid_0's binary_logloss: 0.544802
Acurácia Fold 2: 0.75083

Fold 3
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.551105
[200]	valid_0's binary_logloss: 0.543698
Early stopping, best iteration is:
[180]	valid_0's binary_logloss: 0.540804
Acurácia Fold 3: 0.73422

Acurácia Final: 0.76667
